<html>
<body style="background-color:powderblue;">
<p><i>Author: Shaheer Mansoor</i></p>

## Code Packaging

**This code provides some basic functionality, you can use this to**

1. Read what enviorment the code is running on
2. Read parameters from a json file given your enviorment
3. Log, this is basic logging utility including log file creation. You can also look into python's built in logging as well


In [7]:
# -*- coding: utf-8 -*-
#Define Global Variables here
import_flag = 0
env = 0
cwd = 0
log_file_name = 0
log_hive_table = 0
Live_Run = False

import traceback 

try:

    #Python
    import os
    import sys    
    import json
    import socket    
    import traceback   
    import pandas as pd
    from subprocess import call
    from datetime import datetime, date, time, timedelta         
    
    #Spark ML
    from pyspark.ml import Pipeline, PipelineModel          
    from pyspark.ml.classification import RandomForestClassificationModel
    from pyspark.ml.evaluation import BinaryClassificationEvaluator
    from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit, CrossValidator
    from pyspark.ml.feature import VectorIndexer, StringIndexer, VectorAssembler, OneHotEncoder, Imputer, StandardScaler
    
    #Spark MLLib
    from pyspark.mllib.stat import Statistics  
    
    #Spark Context
    from pyspark.context import SparkContext
    
    #Spark SQL
    from pyspark.sql.session import SparkSession      
    from pyspark.sql import Window, DataFrame
    from pyspark.sql.functions import *   
    from pyspark.sql.functions import lit, col, broadcast, udf             
    from pyspark.sql.types import DoubleType, FloatType, IntegerType, StringType
    
except ImportError as E:
    print("Imports Failed : " + str(E))
    global import_flag
    import_flag = -1

    
#Spark/HIVE Variables
'''
global spark
spark = SparkSession\
        .builder\
        .appName("The RainMaker")\
        .enableHiveSupport()\
        .getOrCreate()
# set the log level to one of ALL, DEBUG, ERROR, FATAL, INFO, OFF, TRACE, WARN (default INFO)
spark.sparkContext.setLogLevel("ERROR")

spark-submit --name RainMaker --executor-memory 5G --master yarn --driver-memory 5G --executor-cores 10 --num-executors 15 --conf spark.dynamicAllocation.minExecutors=10 --conf spark.dynamicAllocation.initialExecutors=10 rainmaker.py 
'''


#Global Optimization Parameters
#Read about other parameters here(https://spark.apache.org/docs/latest/configuration.html#shuffle-behavior)    
spark.conf.set("spark.sql.shuffle.partitions", "10")  
spark.conf.set("spark.default.parallelism", "10")
spark.conf.set("spark.app.name", "The RainMaker")  

#------------------------------------------------------------------------------------------------------------------------------------
#                                              This is the logging fucntion
#------------------------------------------------------------------------------------------------------------------------------------   
def Logger(string):
    #from datetime import datetime
    try:
        if val is None: # The variable
            pass            
    except: # Here we write to the log file
        print("[" + str(datetime.now()) + "] " + str(string))
        with open(log_file_name, "a") as myfile:
            myfile.write("[" + str(datetime.now()) + "] " + str(string) + "\n")
    

#------------------------------------------------------------------------------------------------------------------------------------
#                           This sets up the log file   
#------------------------------------------------------------------------------------------------------------------------------------   
def Set_LogFile():
    #from datetime import date
    try:
        
        global log_file_name
        log_file_name = str(cwd) + "/ModelRun_" + str(date.today()) + ".log"
        return 0
    except:
        print("Errored out during setting up log file")
        traceback.print_exc()
        return -1

#------------------------------------------------------------------------------------------------------------------------------------
#                           This function Tests the JSON File Exists
#------------------------------------------------------------------------------------------------------------------------------------  
def Read_JSON_File():       
    
    r_value = 0    
    try:
        with open(str(cwd) + "/data.json", 'r') as stream:
            y_list = json.load(stream)
            stream.close()
            
    except IOError as exc:
        print(exc)
        traceback.print_exc()
        Logger(str(exc))
        r_value = -1
    
    return r_value

#------------------------------------------------------------------------------------------------------------------------------------
#                           This function can read the JSON file and returns requested values versus a key
#                           This Function also handles the dates for Live and Daignostic Runs
#------------------------------------------------------------------------------------------------------------------------------------  
def Get_From_JSON(env, key):       
    global Live_Run
    r_value = 0
    
    
    try:        
        with open(str(cwd) + "/data.json", 'r') as stream:
            
            j_list = json.load(stream)                             
            return j_list[env][key]    
            
    except ValueError as exc:
        print(exc)      
        traceback.print_exc()
        Logger(str(exc))
        r_value = -1
    except:
        print("Unexpected error on key:" + str(key), sys.exc_info()[0])

        
    stream.close()
    return r_value


#------------------------------------------------------------------------------------------------------------------------------------
#                           Set Enviorment
#------------------------------------------------------------------------------------------------------------------------------------  
def Set_Env():
    try:
        global env
        if "sb-hdp-e" in str(socket.gethostname()):
            env = "ddp"
        if "sb-hdpdev" in str(socket.gethostname()):
            env = "hdp_dev"            
        if "sb-hdppra" in str(socket.gethostname()):
            env = "hdp_prd"
        if "sh-hdpts" in str(socket.gethostname()):
            env = "hdp_test"
        
        return 0
    except Exception as e:    
        Logger("Exception Trace: ", e)
        traceback.print_exc()
        Logger("Errored out during setting up Enviorment")
        Logger(traceback.format_exc())
        return -1


#------------------------------------------------------------------------------------------------------------------------------------
#                           Get Enviorment
#------------------------------------------------------------------------------------------------------------------------------------  
def Get_Env():
    try:
        global env
        return env
    except Exception as e:    
        Logger("Exception Trace: ", e)
        traceback.print_exc()
        Logger("Errored out getting Enviorment")
        Logger(traceback.format_exc())
        return -1
    
    
#------------------------------------------------------------------------------------------------------------------------------------
#                           Helper UDF
#------------------------------------------------------------------------------------------------------------------------------------  
secondelement=udf(lambda v:float(v[1]),FloatType())
    
#------------------------------------------------------------------------------------------------------------------------------------
#                           Create Feature Weights with Names
#------------------------------------------------------------------------------------------------------------------------------------  
def ExtractFeatureImp(featureImp, dataset, featuresCol):
    list_extract = []
    for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    return(varlist.sort_values('score', ascending = False))


#------------------------------------------------------------------------------------------------------------------------------------
#                                     Main Entry Point
#------------------------------------------------------------------------------------------------------------------------------------       

def main():
    try:
        
        print("----------------------------------")
        print("    Starting Model Execution")
        print("----------------------------------")
        print("Trying to import all the necessary libraries required to start a log file")
        
        #Set Working Directory
        global cwd
        cwd = os.getcwd()        
        
        #Set RunDate                     
        Run_Date = str(datetime.strftime(datetime.today(), '%Y-%m-%d')) + ''
        print("RunDate Set to :: " + str(Run_Date))

        #Set Log File 
        has_logfile = Set_LogFile()        
        Logger("Log file is created here " + str(log_file_name)) 
        Logger("Current Working Directory is set to: " + str(cwd)) 

        #Set Environment Name
        has_env = Set_Env()        
        Logger("Current host-name is : " + str(socket.gethostname()) + " and current enviorment is set to: " + str(Get_Env()))         

        #Read JSON File, if this fails the Model will exit
        has_json = Read_JSON_File() 
        
        if has_logfile == 0 and has_json == 0 and has_env == 0:
            Logger("All Good! Starting execution")
            
            #Get Data
            Logger("Getting RainData Set")
            rainDataset_withNulls = spark.table(Get_From_JSON(str(Get_Env()), 'read_db') + '.' + Get_From_JSON(str(Get_Env()), 'Input_Table'))
            Logger("Score Records: {0:,}".format(rainDataset_withNulls.count()))

            for c in rainDataset_withNulls.drop('RainTomorrow', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday').columns:
                rainDataset_withNulls = rainDataset_withNulls.withColumn(str(c), when(trim(rainDataset_withNulls[c]) == 'NA', lit(None)).otherwise(rainDataset_withNulls[c]))

            #Get Data
            Logger("Data Cleansing Step 1: Convert Yes/No to 1/0")
            #Replace Yes/No with 1/0
            rainDataset_withNulls = rainDataset_withNulls.withColumn('RainTomorrow', when(rainDataset_withNulls.RainTomorrow == 'Yes', 1).otherwise(0))

            Logger("Data Cleansing Step 2: Cast Varibles to correct datatypes")
            doubles = ['MinTemp', 'MaxTemp', 'Rainfall','Evaporation', 'Sunshine', 'Pressure9am', 'Pressure3pm', 'Temp9am', 'Temp3pm', 'RISK_MM']
            integers = ['WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Cloud9am', 'Cloud3pm']

            for col in doubles:
                rainDataset_withNulls = rainDataset_withNulls.withColumn(str(col), rainDataset_withNulls[col].cast(DoubleType()))
            Logger("Casted " + str(doubles) + " to doubles")

            for col in integers:
                rainDataset_withNulls = rainDataset_withNulls.withColumn(str(col), rainDataset_withNulls[col].cast(IntegerType()))
            Logger("Casted " + str(integers) + " to integers")

            Logger("Prep Data for Model Step 3: Applying StringIndexers")
            cols = [item[0] for item in rainDataset_withNulls.drop('Date').dtypes if item[1].startswith('string')]

            indexers = [
                StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c), handleInvalid='keep')
                for c in cols
            ]

            pipeline = Pipeline(stages=indexers)

            rainDataset_indexed = pipeline.fit(rainDataset_withNulls).transform(rainDataset_withNulls).drop(*cols)

            Logger("Prep Data for Model Step 3: Applying VectorAssembler")
            #This is to create a list of columns we want as features
            feature_cols = rainDataset_indexed.drop('Date', 'RainTomorrow', 'RISK_MM').columns

            #This is to initialize the VectorAssembler
            assembler_features = VectorAssembler(inputCols=feature_cols, outputCol='features')
            tmp = [assembler_features]
            pipeline = Pipeline(stages=tmp)

            #Fill Nulls with 0's for now, VectorAssembler doesnt take
            rainDataset_indexed = rainDataset_indexed.na.fill(0)

            rainDataset_score = pipeline.fit(rainDataset_indexed).transform(rainDataset_indexed)

            rainDataset_score.cache()
            partitions = rainDataset_score.rdd.getNumPartitions()
            Logger("Score Partitions: {0:,}".format(partitions ))
            Logger("Score Records: {0:,}".format(rainDataset_score.count()))

            Logger("Scoring Step 4: Loading RandomForest Model")  
            rf = RandomForestClassificationModel.load(Get_From_JSON(str(Get_Env()), 'Model_File'))

            Logger("Scoring Samples")  
            predictions = rf.transform(rainDataset_score)
            predictions.cache()
            partitions = rainDataset_score.rdd.getNumPartitions()
            Logger("Scored Partitions: {0:,}".format(partitions ))
            Logger("Scored Records: {0:,}".format(predictions.count()))

            Logger("Save results Step 5: Writing Scores to HIVE")  
            predictions.select('Date', 'Location_indexed', (secondelement('probability')).alias('probability')).createOrReplaceTempView("p901shm_rain_austrialla_scores")
            spark.sql("DROP TABLE IF EXISTS " + str(Get_From_JSON(str(Get_Env()), 'read_db')) + "." + str(Get_From_JSON(str(Get_Env()), 'Output_Table')))
            spark.sql("CREATE TABLE " + str(Get_From_JSON(str(Get_Env()), 'read_db')) + "." + str(Get_From_JSON(str(Get_Env()), 'Output_Table')) + " AS SELECT * FROM p901shm_rain_austrialla_scores")
            
            Logger("All Done! Exiting") 

    except Exception as e:
        print("Errored Out in main")
        print("Exception Trace: ", e)
        traceback.print_exc()
        Logger(traceback.format_exc())
        return -1

if __name__ == "__main__":
    main() 

----------------------------------
    Starting Model Execution
----------------------------------
Trying to import all the necessary libraries required to start a log file
RunDate Set to :: 2019-04-18
[2019-04-18 11:16:56.635314] Log file is created here /data/home/p901shm/projects/Spark-DDP Introduction/ModelRun_2019-04-18.log
[2019-04-18 11:16:56.635520] Current Working Directory is set to: /data/home/p901shm/projects/Spark-DDP Introduction
[2019-04-18 11:16:56.635747] Current host-name is : sb-hdp-e4 and current enviorment is set to: ddp
[2019-04-18 11:16:56.636011] All Good! Starting execution
[2019-04-18 11:16:56.636189] Getting RainData Set
[2019-04-18 11:17:01.973337] Score Records: 142,193
[2019-04-18 11:17:02.799122] Data Cleansing Step 1: Convert Yes/No to 1/0
[2019-04-18 11:17:02.860775] Data Cleansing Step 2: Cast Varibles to correct datatypes
[2019-04-18 11:17:02.963356] Casted ['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine', 'Pressure9am', 'Pressure3pm', 'Te

### Using hostname to define different enviorments

1. We can read host name patterns. Use these to define enviorment specific parameters



In [2]:
print str(Get_Env())

ddp


### Using Json file to get enviorment variables

1. We can use our enviorment variables to pick values from our Json files
2. Ideally we should never hardcode these parameters (especially the ones which change across platforms)


In [3]:
print Get_From_JSON(str(Get_Env()), 'read_db')
print Get_From_JSON(str(Get_Env()), 'Input_Table')

ddp_cvm
p901shm_rain_austrialla_dataset


### Try - Catch

1. We can catch our expections to narrow down what went wrong. This is not important within a notebook enviorment but when we run the models via scripts on other enviorments this can help in debugging
2. We write code in a try block, if something goes wrong the code jumps to catch block. If we use functions to seperate our code blocks we can write exceptions which can be meaningful


In [5]:
from pyspark.sql.functions import to_date

try:
    df = spark.table("ddp_cvm.p901shm_rain_austrialla_scoresd")    
    Logger("All Went Ok!")
    
except Exception as e:      
    Logger("Errored Out on getting table")
    traceback.print_exc()
    Logger(traceback.format_exc())    

[2019-04-18 11:13:14.121229] Errored Out on getting table
[2019-04-18 11:13:14.123385] Traceback (most recent call last):
  File "<ipython-input-5-d66f89e83a55>", line 4, in <module>
    df = spark.table("ddp_cvm.p901shm_rain_austrialla_scoresd")
  File "/usr/hdp/current/spark2-client/python/pyspark/sql/session.py", line 616, in table
    return DataFrame(self._jsparkSession.table(tableName), self._wrapped)
  File "/usr/hdp/current/spark2-client/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 1133, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/hdp/current/spark2-client/python/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
AnalysisException: u"Table or view not found: `ddp_cvm`.`p901shm_rain_austrialla_scoresd`;;\n'UnresolvedRelation `ddp_cvm`.`p901shm_rain_austrialla_scoresd`\n"



Traceback (most recent call last):
  File "<ipython-input-5-d66f89e83a55>", line 4, in <module>
    df = spark.table("ddp_cvm.p901shm_rain_austrialla_scoresd")
  File "/usr/hdp/current/spark2-client/python/pyspark/sql/session.py", line 616, in table
    return DataFrame(self._jsparkSession.table(tableName), self._wrapped)
  File "/usr/hdp/current/spark2-client/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 1133, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/hdp/current/spark2-client/python/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
AnalysisException: u"Table or view not found: `ddp_cvm`.`p901shm_rain_austrialla_scoresd`;;\n'UnresolvedRelation `ddp_cvm`.`p901shm_rain_austrialla_scoresd`\n"


In [6]:
spark.table("ddp_cvm.p901shm_rain_austrialla_scoresss")

AnalysisException: u"Table or view not found: `ddp_cvm`.`p901shm_rain_austrialla_scoresss`;;\n'UnresolvedRelation `ddp_cvm`.`p901shm_rain_austrialla_scoresss`\n"

## Package and Optimize Code

1. You can use the notebook to optimize individual parts of your code
2. Once this is done we can run it within the main, and then create a py file for Spark-Submit


In [29]:
#Get Data
Logger("Getting RainData Set")
rainDataset_withNulls = spark.table(Get_From_JSON(str(Get_Env()), 'read_db') + '.' + Get_From_JSON(str(Get_Env()), 'Input_Table'))
Logger("Score Records: {0:,}".format(rainDataset_withNulls.count()))

for c in rainDataset_withNulls.drop('RainTomorrow', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday').columns:
    rainDataset_withNulls = rainDataset_withNulls.withColumn(str(c), when(trim(rainDataset_withNulls[c]) == 'NA', lit(None)).otherwise(rainDataset_withNulls[c]))

#Get Data
Logger("Data Cleansing Step 1: Convert Yes/No to 1/0")
#Replace Yes/No with 1/0
rainDataset_withNulls = rainDataset_withNulls.withColumn('RainTomorrow', when(rainDataset_withNulls.RainTomorrow == 'Yes', 1).otherwise(0))

Logger("Data Cleansing Step 2: Cast Varibles to correct datatypes")
doubles = ['MinTemp', 'MaxTemp', 'Rainfall','Evaporation', 'Sunshine', 'Pressure9am', 'Pressure3pm', 'Temp9am', 'Temp3pm', 'RISK_MM']
integers = ['WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Cloud9am', 'Cloud3pm']

for col in doubles:
    rainDataset_withNulls = rainDataset_withNulls.withColumn(str(col), rainDataset_withNulls[col].cast(DoubleType()))
Logger("Casted " + str(doubles) + " to doubles")

for col in integers:
    rainDataset_withNulls = rainDataset_withNulls.withColumn(str(col), rainDataset_withNulls[col].cast(IntegerType()))
Logger("Casted " + str(integers) + " to integers")

Logger("Prep Data for Model Step 3: Applying StringIndexers")
cols = [item[0] for item in rainDataset_withNulls.drop('Date').dtypes if item[1].startswith('string')]

indexers = [
    StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c), handleInvalid='keep')
    for c in cols
]

pipeline = Pipeline(stages=indexers)

rainDataset_indexed = pipeline.fit(rainDataset_withNulls).transform(rainDataset_withNulls).drop(*cols)

Logger("Prep Data for Model Step 3: Applying VectorAssembler")
#This is to create a list of columns we want as features
feature_cols = rainDataset_indexed.drop('Date', 'RainTomorrow', 'RISK_MM').columns

#This is to initialize the VectorAssembler
assembler_features = VectorAssembler(inputCols=feature_cols, outputCol='features')
tmp = [assembler_features]
pipeline = Pipeline(stages=tmp)

#Fill Nulls with 0's for now, VectorAssembler doesnt take
rainDataset_indexed = rainDataset_indexed.na.fill(0)

rainDataset_score = pipeline.fit(rainDataset_indexed).transform(rainDataset_indexed)

rainDataset_score.cache()
partitions = rainDataset_score.rdd.getNumPartitions()
Logger("Score Partitions: {0:,}".format(partitions ))
Logger("Score Records: {0:,}".format(rainDataset_score.count()))

Logger("Scoring Step 4: Loading RandomForest Model")  
rf = RandomForestClassificationModel.load(Get_From_JSON(str(Get_Env()), 'Model_File'))

Logger("Scoring Samples")  
predictions = rf.transform(rainDataset_score)
predictions.cache()
partitions = rainDataset_score.rdd.getNumPartitions()
Logger("Scored Partitions: {0:,}".format(partitions ))
Logger("Scored Records: {0:,}".format(predictions.count()))

Logger("Save results Step 5: Writing Scores to HIVE")  
predictions.select('Date', 'Location_indexed', (secondelement('probability')).alias('probability')).createOrReplaceTempView("p901shm_rain_austrialla_scores")
spark.sql("DROP TABLE IF EXISTS " + str(Get_From_JSON(str(Get_Env()), 'read_db')) + "." + str(Get_From_JSON(str(Get_Env()), 'Output_Table')))
spark.sql("CREATE TABLE " + str(Get_From_JSON(str(Get_Env()), 'read_db')) + "." + str(Get_From_JSON(str(Get_Env()), 'Output_Table')) + " AS SELECT * FROM p901shm_rain_austrialla_scores")

[2019-04-17 14:40:21.917710] Getting RainData Set
[2019-04-17 14:40:22.349886] Score Records: 142,193
[2019-04-17 14:40:22.575853] Data Cleansing Step 1: Convert Yes/No to 1/0
[2019-04-17 14:40:22.588577] Data Cleansing Step 2: Cast Varibles to correct datatypes
[2019-04-17 14:40:22.657166] Casted ['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine', 'Pressure9am', 'Pressure3pm', 'Temp9am', 'Temp3pm', 'RISK_MM'] to doubles
[2019-04-17 14:40:22.734683] Casted ['WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Cloud9am', 'Cloud3pm'] to integers
[2019-04-17 14:40:22.735127] Prep Data for Model Step 3: Applying StringIndexers
[2019-04-17 14:40:23.703604] Prep Data for Model Step 3: Applying VectorAssembler
[2019-04-17 14:40:23.892766] Score Partitions: 4
[2019-04-17 14:40:27.360463] Score Records: 142,193
[2019-04-17 14:40:27.361020] Scoring Step 4: Loading RandomForest Model
[2019-04-17 14:40:36.609197] Scoring Samples
[2019-04-17 14:40:38.037217] Sco

DataFrame[]

## Next Steps!

1. Read more about on [spark-submit](https://spark.apache.org/docs/latest/submitting-applications.html)
2. Read about [parameters](https://spark.apache.org/docs/latest/configuration.html) you can configure 